# 📁 File Organizer e Analisi Immagini con Python

## 🔍 Introduzione

Questo progetto in **Python** ha come obiettivo l'organizzazione automatica dei file contenuti in una directory, seguita da un'analisi strutturata delle immagini tramite elaborazione numerica.

Il progetto è suddiviso in **due macro-fasi**:

---

## 🗂️ Step 1: Organizzazione automatica dei file

Lo script scorre **in ordine alfabetico** tutti i file presenti nella cartella principale (`./files`) e li sposta in una **sottocartella tematica**, determinata sulla base del tipo MIME del file (audio, documento o immagine).  
Se la sottocartella non esiste, viene **creata automaticamente**.

### 📌 Funzionalità:

- Determinazione automatica del tipo di file tramite `mimetypes`.
- Creazione dinamica delle cartelle di destinazione.
- Spostamento fisico dei file con tracciamento.
- Generazione (o aggiornamento) di un file `recap.csv` per tenere traccia degli spostamenti effettuati.

### 🧾 Output generato:

Per ogni file trattato, lo script stampa:
- **Nome del file**
- **Tipo MIME**
- **Dimensione (in byte)**
- **Cartella di destinazione**

### 🗃️ Esempio di struttura risultante:


### 📝 Il file `recap.csv`

Viene creato automaticamente se non esistente, e contiene le seguenti colonne:
- **Nome del file**
- **Tipo MIME**
- **Dimensione (in byte)**
---

## 🧪 Step 3: Analisi strutturale delle immagini

Questo step esegue un'**analisi automatica** della cartella `immagini/`, elaborando ogni file tramite la libreria `PIL` (Python Imaging Library) e convertendo ciascuna immagine in un array `NumPy` per l'analisi pixel-based.

### 🔍 Per ogni immagine, vengono estratte:

- **Nome del file**
- **Altezza** (in pixel)
- **Larghezza** (in pixel)
- **Media dei valori di colore**, a seconda del tipo di immagine:
  - Se l'immagine è in scala di grigi (tutti i canali RGB uguali pixel per pixel), viene calcolata la media dei valori del singolo canale e riportata nella colonna `grayscale`.
  - Se l'immagine è a colori (`RGB` o `RGBA`), vengono riportate le medie dei singoli canali: `R`, `G`, `B`, e `ALPHA`.

### 🧾 Esempio di tabella generata:

| name         | height | width | grayscale | R     | G     | B     | ALPHA |
|--------------|--------|-------|-----------|-------|-------|-------|--------|
| bw.png       | 512    | 512   | 124.1     | 0.0   | 0.0   | 0.0   | 0.0    |
| eclipse.png  | 1024   | 768   | 0.0       | 123.6 | 118.7 | 112.5 | 133.0  |
| trump.jpeg   | 800    | 600   | 0.0       | 134.2 | 126.9 | 119.5 | 0.0    |

> **Nota:** Se l'immagine non presenta trasparenza effettiva, il valore della colonna `ALPHA` è impostato a `0.0`.

---

## 🛠️ Librerie utilizzate

- `os`, `shutil` → gestione file e directory
- `mimetypes` → rilevamento tipo file
- `csv` → generazione e aggiornamento log `recap.csv`
- `argparse` → gestione della riga di comando
- `PIL.Image` → lettura ed elaborazione immagini
- `numpy` → elaborazione matriciale
- `tabulate` → visualizzazione tabellare ordinata

---

## ✅ Obiettivi raggiunti

- 🧠 Automazione dell'organizzazione dei file
- 🧾 Tracciabilità degli spostamenti tramite file `.csv`
- 🧬 Analisi automatica e dettagliata dei contenuti immagine
- 📊 Visualizzazione chiara dei risultati tramite tabella formattata

---


### 🧩 Panoramica delle Funzioni

| **Funzione**                           | **Responsabilità**                                                             |
|----------------------------------------|---------------------------------------------------------------------------------|
| `get_mime_type(file_path)`             | Restituisce tipo MIME e dimensione del file in byte                            |
| `classifica_file(mime_type)`           | Ritorna la categoria: `audio`, `immagini`, `documenti`, oppure `None`          |
| `sposta_file(file_path, destinazione)` | Crea la sottocartella se non esiste e vi sposta il file                        |
| `scrivi_log_csv(file_info, csv_path)`  | Aggiunge una riga al file `recap.csv` con nome, tipo e dimensione del file     |
| `organizza_files(folder_path)`         | Funzione principale: scorre i file, li classifica, li sposta e registra il log |
| `analizza_immagini(cartella_immagini)` | Analizza tutte le immagini nella cartella specificata e stampa la tabella      |

---

## 👨‍💻 Autore:

## ***Franco De Giorgio***  

📅 **Data di completamento:** Luglio 2025

---

<H1>STEP 1</H1>

In [1]:
import os           # navigare nella cartella, elencare i file e creare nuove directory 
import shutil       # spostare i file
import mimetypes    # capire il contenuto di un file
import csv          # gestione file recap.csv
# Librerie STEP 3
from PIL import Image  # consente di aprire, analizzare e manipolare immagini in diversi formati (JPEG, PNG, ecc.)
import numpy as np      # per convertire l'immagine in array numerici e calcolare statistiche
from tabulate import tabulate  # genera tabelle in formato testo 

In [2]:
def get_mime_type(file_path):
    """
    Restituisce il tipo MIME di un file, ad esempio 'image/png' o 'document/txt'.
    E la dimensione del file in byte
    Usa il modulo 'mimetypes' di Python che associa estensioni a tipi standardizzati.
    Args:
        file_path: percorso completo al file
    Returns:
        mime_type: stringa del tipo MIME (es. 'image/png')
        size: dimensione in byte
    """
    mime_type, _ = mimetypes.guess_type(file_path)
    if mime_type is None:
        mime_type = 'Tipo sconosciuto'
    size = os.path.getsize(file_path)
    return mime_type, size

In [3]:
def classifica_file(mime_type):
    """
    Classifica il file in base al tipo MIME.
    Ritorna 'audio', 'immagini', 'documenti' oppure None se non riconosciuto.
    """
    if mime_type is None:
        return None
    if mime_type.startswith('audio'):
        return 'audio'
    elif mime_type.startswith('image'):
        return 'immagini'
    elif mime_type.startswith('application') or mime_type.startswith('text'):
        return 'documenti'
    else:
        return None  # tipo sconosciuto

In [4]:
def sposta_file(file_path, folder_path, categoria):
    """
    Sposta il file nella sottocartella corrispondente alla categoria ('audio', 'immagini', 'documenti').
    Crea la sottocartella se non esiste.

    Args:
        file_path: percorso completo del file da spostare
        folder_path: percorso della cartella principale (es. './files')
        categoria: stringa con il nome della categoria
    """
    destinazione_cartella = os.path.join(folder_path, categoria)

    # Crazione cartella categoria se non esiste
    if not os.path.exists(destinazione_cartella):
        os.makedirs(destinazione_cartella)

    # Costruzione nuovo percorso file
    nome_file = os.path.basename(file_path)
    nuovo_percorso = os.path.join(destinazione_cartella, nome_file)

    # Sposta il file nella nuova posizione
    shutil.move(file_path, nuovo_percorso)

    # Percorso file
    return nuovo_percorso 

In [5]:
def organizza_files(folder_path):
    """
    Organizza i file all'interno della cartella:
    - li classifica e li sposta nella rispettiva sottocartella
    - raccoglie le informazioni
    - scrive tutte le righe nel recap.csv in un’unica operazione

    Tutto il ciclo avviene all’interno di un blocco 'with'.
    """
    csv_path = os.path.join(folder_path, 'recap.csv')
    file_esiste = os.path.exists(csv_path)
    intestazioni = ['Nome', 'Tipo', 'Size Byte','Percorso']
    log_data = []  # raccolta delle righe da scrivere

    # apertura unica del file CSV prima del ciclo
    with open(csv_path, mode='a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=intestazioni)

        if not file_esiste:
            writer.writeheader()  # intestazione se file appena creato

        # ciclo principale nel corpo del 'with'
        for filename in sorted(os.listdir(folder_path)):
            file_path = os.path.join(folder_path, filename)

            if not os.path.isfile(file_path) or filename == 'recap.csv':
                continue

            size = os.path.getsize(file_path)
            mime_type, _ = mimetypes.guess_type(file_path)
            categoria = classifica_file(mime_type)

            if categoria is None:
                print(f"[IGNORATO] File non riconosciuto: {filename} (MIME: {mime_type})")
                print("-" * 40)
                continue

            nuovo_percorso = sposta_file(file_path, folder_path, categoria)

            # accumula i dati da scrivere
            file_info = {
                'Nome': filename,
                'Tipo': mime_type,
                'Size Byte': size,
                'Percorso': nuovo_percorso
            }
            log_data.append(file_info)

            # stampa info
            print(f"Nome: {filename}")
            print(f"Tipo: {mime_type}")
            print(f"Dimensione: {size} byte")
            print(f"-> {categoria}/")
            print(f"Percorso aggiornato: {nuovo_percorso}")
            print("-" * 40)

        # scrive tutte le righe raccolte in una sola operazione
        if log_data:
            writer.writerows(log_data)


In [6]:
# ESEGUE LO SCRIPT
organizza_files('./files')

<H1>STEP 3</H1>

In [7]:
def analizza_immagini(cartella_immagini='./files/immagini'):
    """
    Analizza tutte le immagini nella cartella specificata e stampa una tabella con:
    - nome del file
    - altezza e larghezza
    - media dei canali colore (grayscale o RGB / RGBA)
    
    Questa funzione rileva automaticamente la modalità dell'immagine
    ottimizzando l'elaborazione per diversi tipi di immagini (grigi, colori, con trasparenza).
    """
    risultati = [] # Lista dati di ogni immagine
    
    # Verifica che la cartella specificata esista e sia accessibile     
    if not os.path.isdir(cartella_immagini):
        print(f"Errore: la cartella '{cartella_immagini}' non esiste o non è accessibile.")
        return # Termina la funzione se la cartella non è valida.

    # Ordina i nomi dei file e itererà su ogni elemento all'interno della cartella specificata
    for filename in sorted(os.listdir(cartella_immagini)):
        percorso_file = os.path.join(cartella_immagini, filename) # Costruisce il percorso completo del file.

        # Ignora le directory e i file che non sono immagini valide 
        if not os.path.isfile(percorso_file):
            continue

        try:
            # Apre il file immagine utilizzando la libreria Pillow.
            with Image.open(percorso_file) as img:
                # Converte l'immagine in un formato NumPy adatto all'analisi.
                # Converte( '1', 'P') in formati più gestibili ('L' o 'RGB')
                # mantiene le modalità 'standard' ('L', 'RGB', 'RGBA')                
                array = None # Inizializziamo l'array a None
                
                if img.mode == 'L': # Immagine già in scala di grigi (1 canale per Pillow, 2D per NumPy).
                    array = np.array(img)
                elif img.mode == 'RGB': # Immagine a colori senza canale alpha (3 canali, 3D per NumPy).
                    array = np.array(img)
                elif img.mode == 'RGBA': # Immagine a colori con canale alpha (4 canali, 3D per NumPy).
                    array = np.array(img)
                elif img.mode == '1' or img.mode == 'P': # Immagini binarie (solo bianco/nero) o con palette.            
                    # provo prima la conversione a 'L'
                    try:
                        array = np.array(img.convert('L')) 
                    except ValueError:
                        # Se la conversione a 'L' fallisce (es. una palette che contiene colori veri e non solo grigi),
                        # provo  a convertire a 'RGB'
                        print(f"[AVVISO] Impossibile convertire '{filename}' (mode: {img.mode}) a 'L'. Tentativo a 'RGB'.")
                        array = np.array(img.convert('RGB'))
                else:
                    # Gestiamo eventuali altre modalità meno comuni                    
                    print(f"[AVVISO] Modalità immagine '{img.mode}' di '{filename}' non gestita esplicitamente. Conversione a RGB.")
                    array = np.array(img.convert('RGB'))
                
                # Un controllo di sicurezza: se per qualche motivo l'array NumPy non è stato creato, solleviamo un errore.
                if array is None:
                    raise ValueError(f"Impossibile ottenere un array NumPy da immagine '{filename}' con modalità '{img.mode}'.")

                # Ottiene le dimensioni dell'immagine dall'oggetto Pillow
                altezza, larghezza = img.height, img.width
            
                # Inizializziamo tutte le variabili a 0.0
                grayscale_val = 0.0 
                r, g, b = 0.0, 0.0, 0.0
                alpha_val = 0.0 

                # Analisi Basata sulla Dimensionalità dell'Array NumPy 
                if array.ndim == 2:                    
                    # L'array 2D ha solo un canale (intensità di grigio).
                    grayscale_val = np.mean(array) # Calcola la media di tutti i pixel dell'unico canale.                    
                elif array.ndim == 3:
                    # Immagini con canali colore (RGB, RGBA)
                    num_channels = array.shape[2] # Ottiene il numero effettivo di canali (3 per RGB, 4 per RGBA).

                    if num_channels >= 3:
                        # Calcola le medie dei primi 3 canali                        
                        mean_rgb_channels = np.mean(array[:, :, :3], axis=(0, 1))
                        
                        # Verifica se l'immagine a colori è in realtà una scala di grigi "simulata".                        .
                        is_simulated_grayscale = np.allclose(mean_rgb_channels[0], mean_rgb_channels[1]) and \
                                                 np.allclose(mean_rgb_channels[1], mean_rgb_channels[2])

                        if is_simulated_grayscale:
                            # Se è una scala di grigi simulata, riempi solo la colonna 'grayscale'.
                            grayscale_val = mean_rgb_channels[0] 
                        else:
                            # Se è a colori, riempi le colonne R, G, B.
                            r = mean_rgb_channels[0]
                            g = mean_rgb_channels[1]
                            b = mean_rgb_channels[2]
                        
                        # Gestione del Canale Alpha, se l'immagine ha 4 canali (RGBA), analizza il canale alpha.
                        if num_channels == 4:
                            mean_alpha_channel = np.mean(array[:, :, 3]) # Media dei valori del canale alpha.
                            # Se la media del canale alpha è inferiore a 255 (massima opacità), significa che c'è
                            # un grado di trasparenza reale, quindi registriamo il valore. Altrimenti, rimane 0.0.
                            if mean_alpha_channel < 255:
                                alpha_val = mean_alpha_channel
                            
                # Aggiunge tutti i dati raccolti per il file corrente alla lista dei risultati.
                risultati.append([filename, altezza, larghezza, grayscale_val, r, g, b, alpha_val])

        
        # Se si verifica un errore durante l'apertura o l'elaborazione di un file stampiamo un messaggio e passiamo al file successivo.
        except Exception as e:
            print(f"[ERRORE] Impossibile aprire o elaborare {filename}: {e}")
            continue

    # Stampa la tabella 
    intestazioni = ['name', 'height', 'width', 'grayscale', 'R', 'G', 'B', 'ALPHA']
    print(tabulate(risultati, headers=intestazioni, tablefmt='grid', floatfmt='.1f'))

In [ ]:
# ESEGUE LO SCRIPT
analizza_immagini()